In [ ]:
import os 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from pymatgen.core import Structure
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)


In [ ]:
structure=Structure.from_file("../data/structure/9006189.cif")
structure.add_oxidation_state_by_site([3]*8+[2]*8+[3]*8+[-2]*32)
structure.add_site_property('charge',[2.295]*8+[1.530]*8+[2.295]*8+[-1.530]*32)
unit_cell_atoms=len(structure.sites)
structure.add_site_property('atom_number',np.arange(unit_cell_atoms))

site_map={"Fe2+":"$Fe^2_+$","Fe3+":"$Fe^3_+$","O2-":"$O^2_-$"}
data_list=list()
for i, siteA in enumerate(structure.sites):
    for j, siteB in enumerate(structure.sites):
        if siteA==siteB:
            continue
        r=siteA.distance(siteB)
        data_list.append({"Pair":"%s:%s"%(site_map[siteA.species_string],site_map[siteB.species_string]),"r (A)":np.round(r,1)})
df_unit_cell_dist=pd.DataFrame(data_list).drop_duplicates().reset_index(drop=True)


In [ ]:
results_directory={"../results/lammps/interactions/lennard_jones/":"LJ"
               ,"../results/lammps/interactions/coulombic_buckingham/":"CB"
               ,"../results/lammps/interactions/reaxff2010_ox/":"ReaxFF 2010 ox"
               ,"../results/lammps/interactions/reaxff2022/":"ReaxFF 2022"
              }
df_list=list()

atom_map={1:"$Fe^2_+$",2:"$Fe^3_+$",3:"$O^2_-$"}
for directory,force_field in results_directory.items():
    for dataset in os.listdir(directory):
        if not "pair" in dataset:
            continue
        df_tmp=pd.read_csv(os.path.join(directory,dataset,"results/force.csv"),names=["index","atom","r (A)","y","z","F(r) (eV/A)","fy","fz"],skiprows=2)
        df_tmp["Force Field"]=force_field
        if force_field in ["LJ",""]:
            df_tmp["F(r) (eV/A)"]*=0.0104
        idx1=df_tmp["index"]==1
        idx2=df_tmp["index"]==2
        df_tmp["Pair"]="%s:%s"%(atom_map[df_tmp[idx1]["atom"].unique()[0]],atom_map[df_tmp[idx2]["atom"].unique()[0]])
        df_tmp=df_tmp.query("index==2")
        df_tmp2=pd.read_csv(os.path.join(directory,dataset,"results/E_pair.txt"),names=["V(r) (eV)"])
        df_tmp["V(r) (eV)"]=df_tmp2.values
        df_list.append(df_tmp)
    
df_f=pd.concat(df_list,ignore_index=True)


In [ ]:
fig, axs = plt.subplots(ncols=3,nrows=2,figsize=(7*3,4*2))
sns.set_context("talk")

pairs=df_f["Pair"].unique()

labx=0.8
laby=0.9

ylim_min=-10
ylim_max=250
rmin=1

ax=axs[0,0]
ax.text(labx, laby, '(a)',weight='bold' ,transform=ax.transAxes)
sns.lineplot(data=df_f.query("`r (A)` > %f and `Pair` == '%s'"%(rmin,pairs[0]))
            ,y='V(r) (eV)'
            ,x='r (A)'
            ,hue='Force Field'
            ,ax=ax
            ,palette='dark'
       )
sns.move_legend(ax,'lower center',bbox_to_anchor=(0.5,0.95),ncol=6,bbox_transform=fig.transFigure)
ax.spines['bottom'].set_color('0')
ax.spines['top'].set_color('1')
ax.spines['right'].set_color('1')
ax.spines['left'].set_color('0')
ax.tick_params(direction='out', width=3, bottom=True, left=True)
ax.grid(False)
ymin=-50
ax.set_ylim(ymin,200)
ax.set_title(pairs[0])

idx=df_unit_cell_dist["Pair"]==pairs[0]
df_tmp=df_unit_cell_dist[idx].query("`r (A)` < %f"%df_f["r (A)"].max()).copy()
df_tmp["norm"]=15+ymin
sns.scatterplot(data=df_tmp
                 ,x="r (A)"
                 ,y="norm"
                ,legend=False
                ,marker="^"
                ,ax=ax
            )
ax.xaxis.set_minor_locator(MultipleLocator(0.25))
ax.yaxis.set_minor_locator(MultipleLocator(25))
ax.tick_params(which='minor', length=4, width=3)

ax.set_xlabel("")
ax.set_ylabel("Energy (meV)")


########################## --- (b) --- ##########################
ax=axs[0,1]
ax.text(labx, laby, '(b)',weight='bold' ,transform=ax.transAxes)
sns.lineplot(data=df_f.query("`r (A)` > %f and `Pair` == '%s'"%(rmin,pairs[3]))
            ,y='V(r) (eV)'
            ,x='r (A)'
            ,hue='Force Field'
            ,ax=ax
            ,palette='dark'
             ,legend=False
       )
ax.spines['bottom'].set_color('0')
ax.spines['top'].set_color('1')
ax.spines['right'].set_color('1')
ax.spines['left'].set_color('0')
ax.tick_params(direction='out', width=3, bottom=True, left=True)
ax.grid(False)
ymin=-50
ax.set_ylim(ymin,300)
ax.set_title(pairs[3])

idx=df_unit_cell_dist["Pair"]==pairs[3]
df_tmp=df_unit_cell_dist[idx].query("`r (A)` < %f"%df_f["r (A)"].max()).copy()
df_tmp["norm"]=15+ymin
sns.scatterplot(data=df_tmp
                 ,x="r (A)"
                 ,y="norm"
                ,legend=False
                ,marker="^"
                ,ax=ax
            )
ax.xaxis.set_minor_locator(MultipleLocator(0.25))
ax.yaxis.set_minor_locator(MultipleLocator(25))
ax.tick_params(which='minor', length=4, width=3)

ax.set_xlabel("")
ax.set_ylabel("")

########################## --- (c) --- ##########################
ax=axs[0,2]
ax.text(labx, laby, '(c)',weight='bold' ,transform=ax.transAxes)
sns.lineplot(data=df_f.query("`r (A)` > %f and `Pair` == '%s'"%(rmin,pairs[1]))
            ,y='V(r) (eV)'
            ,x='r (A)'
            ,hue='Force Field'
            ,ax=ax
            ,palette='dark'
             ,legend=False
       )
ax.spines['bottom'].set_color('0')
ax.spines['top'].set_color('1')
ax.spines['right'].set_color('1')
ax.spines['left'].set_color('0')
ax.tick_params(direction='out', width=3, bottom=True, left=True)
ax.grid(False)
ymin=-50
ax.set_ylim(ymin,400)
ax.set_title(pairs[1])


idx=df_unit_cell_dist["Pair"]==pairs[1]
df_tmp=df_unit_cell_dist[idx].query("`r (A)` < %f"%df_f["r (A)"].max()).copy()
df_tmp["norm"]=15+ymin
sns.scatterplot(data=df_tmp
                 ,x="r (A)"
                 ,y="norm"
                ,legend=False
                ,marker="^"
                ,ax=ax
            )
ax.xaxis.set_minor_locator(MultipleLocator(0.25))
ax.yaxis.set_minor_locator(MultipleLocator(25))
ax.tick_params(which='minor', length=4, width=3)

ax.set_xlabel("")
ax.set_ylabel("")

########################## --- (d) --- ##########################
ax=axs[1,0]
ax.text(labx, laby, '(d)',weight='bold' ,transform=ax.transAxes)
sns.lineplot(data=df_f.query("`r (A)` > %f and `Pair` == '%s'"%(rmin,pairs[4]))
            ,y='V(r) (eV)'
            ,x='r (A)'
            ,hue='Force Field'
            ,ax=ax
            ,palette='dark'
             ,legend=False
       )
ax.spines['bottom'].set_color('0')
ax.spines['top'].set_color('1')
ax.spines['right'].set_color('1')
ax.spines['left'].set_color('0')
ax.tick_params(direction='out', width=3, bottom=True, left=True)
ax.grid(False)
ymin=-300
ax.set_ylim(ymin,100)
ax.set_title(pairs[4])

idx=df_unit_cell_dist["Pair"]==pairs[4]
df_tmp=df_unit_cell_dist[idx].query("`r (A)` < %f"%df_f["r (A)"].max()).copy()
df_tmp["norm"]=15+ymin
sns.scatterplot(data=df_tmp
                 ,x="r (A)"
                 ,y="norm"
                ,legend=False
                ,marker="^"
                ,ax=ax
            )

ax.xaxis.set_minor_locator(MultipleLocator(0.25))
ax.yaxis.set_minor_locator(MultipleLocator(25))
ax.tick_params(which='minor', length=4, width=3)

ax.set_xlabel("Separation (Å)")
ax.set_ylabel("Energy (meV)")

########################## --- (e) --- ##########################
ax=axs[1,1]
ax.text(labx, laby, '(e)',weight='bold' ,transform=ax.transAxes)
sns.lineplot(data=df_f.query("`r (A)` > %f and `Pair` == '%s'"%(rmin,pairs[2]))
            ,y='V(r) (eV)'
            ,x='r (A)'
            ,hue='Force Field'
            ,ax=ax
            ,palette='dark'
             ,legend=False
       )
ax.spines['bottom'].set_color('0')
ax.spines['top'].set_color('1')
ax.spines['right'].set_color('1')
ax.spines['left'].set_color('0')
ax.tick_params(direction='out', width=3, bottom=True, left=True)
ax.grid(False)
ymin=-200
ax.set_ylim(ymin,200)
ax.set_title(pairs[2])

idx=df_unit_cell_dist["Pair"]==pairs[2]
df_tmp=df_unit_cell_dist[idx].query("`r (A)` < %f"%df_f["r (A)"].max()).copy()
df_tmp["norm"]=15+ymin
sns.scatterplot(data=df_tmp
                 ,x="r (A)"
                 ,y="norm"
                ,legend=False
                ,marker="^"
                ,ax=ax
            )
ax.xaxis.set_minor_locator(MultipleLocator(0.25))
ax.yaxis.set_minor_locator(MultipleLocator(25))
ax.tick_params(which='minor', length=4, width=3)

ax.set_xlabel("Separation (Å)")
ax.set_ylabel("")


########################## --- (f) --- ##########################
ax=axs[1,2]
ax.text(labx, laby, '(f)',weight='bold' ,transform=ax.transAxes)
sns.lineplot(data=df_f.query("`r (A)` > %f and `Pair` == '%s'"%(rmin,pairs[5]))
            ,y='V(r) (eV)'
            ,x='r (A)'
            ,hue='Force Field'
            ,ax=ax
            ,palette='dark'
             ,legend=False
       )
ax.spines['bottom'].set_color('0')
ax.spines['top'].set_color('1')
ax.spines['right'].set_color('1')
ax.spines['left'].set_color('0')
ax.tick_params(direction='out', width=3, bottom=True, left=True)
ax.grid(False)

ymin=-150
ax.set_ylim(ymin,200)
ax.set_title(pairs[5])

idx=df_unit_cell_dist["Pair"]==pairs[5]
df_tmp=df_unit_cell_dist[idx].query("`r (A)` < %f"%df_f["r (A)"].max()).copy()
df_tmp["norm"]=10+ymin
sns.scatterplot(data=df_tmp
                 ,x="r (A)"
                 ,y="norm"
                ,legend=False
                ,marker="^"
                ,ax=ax
            )
ax.xaxis.set_minor_locator(MultipleLocator(0.25))
ax.yaxis.set_minor_locator(MultipleLocator(25))
ax.tick_params(which='minor', length=4, width=3)

ax.set_xlabel("Separation (Å)")
ax.set_ylabel("")

# set the spacing between subplots
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.175, 
                    hspace=0.3
                   )

plt.savefig("../figures/fig2.pdf", pad_inches=0.2,bbox_inches="tight")